<center><h1>Swamy_Meghana_HW3</h1></center>
<br>
<br>

Name: Meghana Putta Swamy
<br>
Github Username: meghanaswamy1995
<br> 
USC ID: 6565038173 

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import urllib
import operator
import sys
import warnings
import pandas as pd
import numpy as np
import seaborn as sbn 
import matplotlib.pyplot as plt
import bootstrapped.bootstrap as bs
import scipy.stats as stats
import bootstrapped.stats_functions as bs_stats
from sklearn import preprocessing
from beautifultable import BeautifulTable
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import roc_curve, confusion_matrix, roc_auc_score, auc
from sklearn.feature_selection import chi2
from imblearn.over_sampling import SMOTE 
warnings.simplefilter(action='ignore', category=FutureWarning) 

In [2]:
# arrange data in a table
def printDatainTable(tableHeaders, tableData):
    table = BeautifulTable(maxwidth=100)
    table.column_headers = tableHeaders
    for i in range(0, len(tableData)):
        table.append_row(tableData[i])
    print(table)

In [3]:
def readData(fileURL):
    data = []
    inputFile = open(fileURL, "r")
    #inputFile = urllib.request.urlopen(fileURL)
    cnt = 1
    for line in inputFile:
        if cnt<=5:
            cnt += 1
            continue
        else:
            columnData = line.split(',')
            if len(columnData)==1:
                columnData = line.split(' ')
            for j in range(1, len(columnData)):
                columnData[j] = columnData[j].replace('\n', '')
                columnData[j] = columnData[j].strip()
                if len(columnData[j])==0:
                    columnData.pop(j)
                    continue
                columnData[j] = float(columnData[j])
            data.append(columnData[1:])
    
    data = pd.DataFrame(data, columns=columnNames)
    return data 

In [4]:
def getStatistics(data):
    statistics = data.describe()
    stats = {}
    stats['Mean'] = statistics[1:2].values[0].tolist()
    stats['Std'] = statistics[2:3].values[0].tolist()
    stats['Median'] = statistics[5:6].values[0].tolist()
    stats['Min'] = statistics[3:4].values[0].tolist()
    stats['Max'] = statistics[7:8].values[0].tolist()
    stats['FirstQuartile'] = statistics[4:5].values[0].tolist()
    stats['ThirdQuartile'] = statistics[6:7].values[0].tolist()
    return stats

Get the AReM Data Set

In [5]:
directoryToRead = ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']
columnNames = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
featureNames = []

directoryName = 'AReM'
baseURL = '../data/'

trainDataUpperBound = [7, 6, 15, 15, 15, 15, 15]
testDataUpperBound = [2, 2, 3, 3, 3, 3, 3]

### (b) Test and Train Data

Create Test Data

In [6]:
data = {}
         
for i in range(0, len(directoryToRead)):
    for j in range(1, trainDataUpperBound[i]+1):
        fileName = directoryName + '/' + directoryToRead[i] + '/dataset' + str(j) + '.csv' 
        fileUrl = baseURL + fileName
        data[fileName] = readData(fileUrl)

            
for k,v in data.items():
    print('--------------------------------------------')
    print('File Name:', k)
    print('-------------------------------------------')
    print('Total observations:', len(v))
    print('-------------------------------------------')
    print('Total features:', len(v.columns))
    print('-------------------------------------------')
    print('First 5 observations:')
    print('-------------------------------------------')
    print(v.head()) 
    print() 

--------------------------------------------
File Name: AReM/bending1/dataset1.csv
-------------------------------------------
Total observations: 480
-------------------------------------------
Total features: 6
-------------------------------------------
First 5 observations:
-------------------------------------------
   avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  var_rss23
0      39.25       0.43      22.75       0.43      33.75        1.3
1      39.25       0.43      23.00       0.00      33.00        0.0
2      39.25       0.43      23.25       0.43      33.00        0.0
3      39.50       0.50      23.00       0.71      33.00        0.0
4      39.50       0.50      24.00       0.00      33.00        0.0

--------------------------------------------
File Name: AReM/bending1/dataset2.csv
-------------------------------------------
Total observations: 480
-------------------------------------------
Total features: 6
-------------------------------------------
First 5 obs

In [7]:
trainData = {}

for i in range(0, len(directoryToRead)):
    start = testDataUpperBound[i]+1
    end = trainDataUpperBound[i]
    for j in range(start, end+1):
        fileName = directoryName + '/' + directoryToRead[i] + '/dataset' + str(j) + '.csv'
        trainData[fileName] = data[fileName]

for k,v in trainData.items():
    print('-------------------------------------------')
    print('File Name:', k)
    print('-------------------------------------------')
    print('Total observations:', len(v))
    print('-------------------------------------------')
    print('Total features:', len(v.columns))
    print('-------------------------------------------')
    print('First 5 observations:')
    print('-------------------------------------------')
    print(v.head())
    print()

-------------------------------------------
File Name: AReM/bending1/dataset3.csv
-------------------------------------------
Total observations: 480
-------------------------------------------
Total features: 6
-------------------------------------------
First 5 observations:
-------------------------------------------
   avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  var_rss23
0      42.00       0.71      21.25       0.43      30.00       0.00
1      41.50       0.50      20.25       1.48      31.25       1.09
2      41.50       0.50      14.25       1.92      33.00       0.00
3      40.75       0.83      15.75       0.43      33.00       0.00
4      40.00       0.71      20.00       2.74      32.75       0.43

-------------------------------------------
File Name: AReM/bending1/dataset4.csv
-------------------------------------------
Total observations: 480
-------------------------------------------
Total features: 6
-------------------------------------------
First 5 obser

### (c) Feature Extraction

#### i. Research

Time-domain features used in time-series classification.
Statistical Features:
a. Mean
b. Median
c. Standard Deviation
d. Skewness, Kurtosis
e. Minimum and Maximum

Time Series Analysis Related Features:
a. Cross Correlation
b. Auto Correlation
c. Parameters of Autoregressive (AR) part of Autoregressive Integrated Moving Average (ARIMA) model
d. Parameters of Moving Average (MA) part of ARIMA model
e. Orders of AR, Integrated (I) and MA part of ARIMA model

#### ii. Extraction

In [8]:
timeDomainData = []
for k,v in data.items():
    activity = k.split('/')[1]
    stats = getStatistics(v)
    means = stats['Mean']
    stds = stats['Std']
    medians = stats['Median']
    minValues = stats['Min']
    maxValues = stats['Max']
    firstQuartiles = stats['FirstQuartile']
    thirdQuartiles = stats['ThirdQuartile']
    
    if len(featureNames)==0:
        for i in range(0, len(columnNames)):
            featureNames.append('Min_' + columnNames[i])
            featureNames.append('Max_' + columnNames[i])
            featureNames.append('Mean_' + columnNames[i])
            featureNames.append('Median_' + columnNames[i])
            featureNames.append('Std_' + columnNames[i])
            featureNames.append('FirstQuartile_' + columnNames[i])
            featureNames.append('ThirdQuartile_' + columnNames[i])
    
    row = []
    for i in range(0, len(columnNames)):
        row.append(minValues[i])
        row.append(maxValues[i])
        row.append(means[i])
        row.append(medians[i])
        row.append(stds[i])
        row.append(firstQuartiles[i])
        row.append(thirdQuartiles[i])
    row.append(activity)
    timeDomainData.append(row)
     
timeDomainDataColumns = []
timeDomainDataColumns = featureNames[:]
timeDomainDataColumns.append('Activity')
timeDomainData = pd.DataFrame(timeDomainData, columns=timeDomainDataColumns)

print('--------------------------------------------------')
print('Total observations in time-domain feature data:', len(timeDomainData))
print('--------------------------------------------------')
print('Total features in time-domain feature data:', len(featureNames))
print('--------------------------------------------------')
print('First 5 observations in time-domain feature data:')
print('--------------------------------------------------')
print(timeDomainData[featureNames].head())

--------------------------------------------------
Total observations in time-domain feature data: 88
--------------------------------------------------
Total features in time-domain feature data: 42
--------------------------------------------------
First 5 observations in time-domain feature data:
--------------------------------------------------
   Min_avg_rss12  Max_avg_rss12  Mean_avg_rss12  Median_avg_rss12  \
0          37.25          45.00       40.624792             40.50   
1          38.00          45.67       42.812812             42.50   
2          35.00          47.40       43.954500             44.33   
3          33.00          47.75       42.179813             43.50   
4          33.00          45.75       41.678063             41.75   

   Std_avg_rss12  FirstQuartile_avg_rss12  ThirdQuartile_avg_rss12  \
0       1.476967                    39.25                    42.00   
1       1.435550                    42.00                    43.67   
2       1.558835       

#### iii. Standard Deviation

In [9]:
stats = timeDomainData[featureNames].describe()
stds = stats[2:3].values[0].tolist()
tableHeaders = ['Feature Name', 'Standard Deviation']
tableData = []
for i in range(0, len(featureNames)):
    row = []
    row.append(featureNames[i])
    row.append(stds[i])
    tableData.append(row)

print('STANDARD DEVIATION OF TIME-DOMAIN FEATURES:')
printDatainTable(tableHeaders, tableData)

STANDARD DEVIATION OF TIME-DOMAIN FEATURES:
+-------------------------+--------------------+
|      Feature Name       | Standard Deviation |
+-------------------------+--------------------+
|      Min_avg_rss12      |        9.57        |
+-------------------------+--------------------+
|      Max_avg_rss12      |       4.394        |
+-------------------------+--------------------+
|     Mean_avg_rss12      |       5.336        |
+-------------------------+--------------------+
|    Median_avg_rss12     |        5.44        |
+-------------------------+--------------------+
|      Std_avg_rss12      |       1.772        |
+-------------------------+--------------------+
| FirstQuartile_avg_rss12 |       6.154        |
+-------------------------+--------------------+
| ThirdQuartile_avg_rss12 |       5.139        |
+-------------------------+--------------------+
|      Min_var_rss12      |        0.0         |
+-------------------------+--------------------+
|      Max_var_rss12     

#### 90% bootstrap confidence interval for standard deviation of each feature.

In [10]:
tableData = []
for i in range(0, len(featureNames)):
    row = []
    samples = np.array(timeDomainData[featureNames[i]])
    bootResult = bs.bootstrap(samples, stat_func=bs_stats.std, alpha=0.1)
    row.append(featureNames[i])
    row.append(stds[i])
    row.append(bootResult.lower_bound)
    row.append(bootResult.upper_bound)
    tableData.append(row)
    
print('STANDARD DEVIATION & 90% CONFIDENCE INTERVAL OF TIME-DOMAIN FEATURES:')
tableHeader = ['Feature Name', 'Standard Deviation', 'Lower Bound', 'Upper Bound']
printDatainTable(tableHeader, tableData)

STANDARD DEVIATION & 90% CONFIDENCE INTERVAL OF TIME-DOMAIN FEATURES:
+-------------------------+--------------------+-------------+-------------+
|      Feature Name       | Standard Deviation | Lower Bound | Upper Bound |
+-------------------------+--------------------+-------------+-------------+
|      Min_avg_rss12      |        9.57        |    8.318    |   10.797    |
+-------------------------+--------------------+-------------+-------------+
|      Max_avg_rss12      |       4.394        |    3.486    |    5.423    |
+-------------------------+--------------------+-------------+-------------+
|     Mean_avg_rss12      |       5.336        |    4.757    |    5.919    |
+-------------------------+--------------------+-------------+-------------+
|    Median_avg_rss12     |        5.44        |    4.852    |    6.056    |
+-------------------------+--------------------+-------------+-------------+
|      Std_avg_rss12      |       1.772        |    1.587    |    1.964    |
+-----

#### iv. Select Features

1. Mean: It is the best measure of central tendency when the data distribution is continuous.
2. Median: It is a good feature of central tendency that is less affected by outliers and skewed data.
3. Standard Deviation: It is an ideal measure of dispersion. 

## 2. ISLR 3.7.4

### (a) Linear Train

(a) Because it may provide a more precise fit to data that matched with a wider irreducible error (Var(epsilon)), I would anticipate that the polynomial regression would have a lower training RSS than the linear regression.


### (b) Linear Test

(b) Contrary to (a), I would anticipate that the polynomial regression would have a higher test RSS than the linear regression since the overfit from training would introduce more error.

### (c) Not Linear Train

(c) Due to its greater flexibility, polynomial regression has lower train RSS than linear fit because it will produce closer follow points regardless of the underlying true relationship.


### (d) Not Linear Testing

(d) Given that the issue description states that it is impossible to determine "how far it is from linear," there is insufficient data to determine which test RSS would be lower for either regression. The linear regression test RSS could be lower than the cubic regression test RSS if it is more linear than cubic.
The cubic regression test RSS could also be lower than the linear regression test RSS if it is more cubical than linear. Due to the bias-variance trade-off, it is unclear what degree of flexibility will better suit the data.

## 3. ISLR 3.7.3 - Extra Practice 

3. Y = 50 + 20(gpa) + 0.07(iq) + 35(gender) + 0.01(gpa * iq) - 10 (gpa * gender)

    (a) Y = 50 + 20 k_1 + 0.07 k_2 + 35 gender + 0.01(k_1 * k_2) - 10 (k_1 * gender)
    
    male: (gender = 0)   50 + 20 k_1 + 0.07 k_2 + 0.01(k_1 * k_2)
    
    female: (gender = 1) 50 + 20 k_1 + 0.07 k_2 + 35 + 0.01(k_1 * k_2) - 10 (k_1)

    Once the GPA is high enough, males earn more on average. => 
    

    (b) Y(Gender = 1, IQ = 110, GPA = 4.0)
    = 50 + 20 * 4 + 0.07 * 110 + 35 + 0.01 (4 * 110) - 10 * 4
    = 137.1

    (c) False. To evaluate whether or not the interaction term is statistically significant, we must look at the p-value of the regression coefficient. 
